In [1]:
from keras.models  import load_model
import keras.backend as K

import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer

from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np

from support import norm

aspects = ['bit', 'domain']

Using TensorFlow backend.


[]


In [2]:
vec = cPickle.load(open('../../yelpdata/total_vec_120K_embed.p', 'rb'))

In [3]:
import pandas as pd
df = pd.read_csv('../../yelpdata/total_data_120K.csv')

In [4]:
model = load_model('../store/weights/food/WedNov816:36:412017/loss.h5', 
                   custom_objects={'contrastive_loss' : lambda a,b:b})

In [5]:
model_c1 = {}
for aspect in aspects :
    model_c1[aspect] = K.function(model.get_layer('pred_'+aspect).get_layer('pool_'+aspect).inputs, 
                          [model.get_layer('pred_'+aspect).get_layer('pool_'+aspect).get_output_at(0)])

In [6]:
ds = df
from sklearn.model_selection import train_test_split
ds['bit'] = ds['stars'].apply(lambda x : x > 3.0)
sample, sample_2 = train_test_split(ds.index, stratify=ds[['bit', 'domain']], train_size=0.1)

/home/sarthak/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [23]:
vecs = {k:[] for k in aspects}
i, bs = 0, 64
while i*bs < len(sample):
    for aspect in aspects :
        samples = sample[i*bs:(i+1)*bs]
        result = model_c1[aspect]([vec.X[samples], 0])[0]
        vecs[aspect].append(result)
    i += 1
for aspect in aspects :
    result = np.concatenate(vecs[aspect], axis=0)
    vecs[aspect] = result #normalize(result, 'l2')

In [28]:
from sklearn.decomposition import PCA, TruncatedSVD
pca_models = {}
pcas = {}

for aspect in aspects :
    pca = TruncatedSVD(n_components=2)
    pcas[aspect] = pca.fit_transform(vecs[aspect])
    pca_models[aspect] = pca
    
# for aspect in aspects :
#     pcas[aspect] = pca_models[aspect].transform(vecs[aspect])

ValueError: array must not contain infs or NaNs

In [ ]:
import matplotlib.pyplot as plt
for aspect in aspects :
    plt.scatter(pcas[aspect][:, 0], pcas[aspect][:, 1])
    #plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.title(aspect)
    plt.show()

In [ ]:
nb_studies = len(da)
H = np.zeros((nb_studies, nb_studies))
for i in range(nb_studies) :
    H[i, da[da['IM_population'] == da['IM_population'][i]].index] = 1
np.fill_diagonal(H, 0)

scores = {}
for aspect in aspects :
    scores[aspect] = np.dot(norm_pool_da[aspect], norm_pool_da[aspect].T)
    np.fill_diagonal(scores[aspect], -1000)

In [ ]:
from sklearn.metrics import roc_auc_score
series = {}
for aspect in aspects :
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[i], scores[aspect][i])
    rocs = {}
    for key in im_map :
        idxs = da[da['IM_population'] == key].index
        rocs[key] = np.mean(np.array(aucs)[idxs])
    rocs['mean'] = np.mean(aucs)
    series[aspect] = pd.Series(rocs, name=aspect)
pd.DataFrame(series.values())

In [ ]:
get_ipython().system('cp "Decision Aids Single.ipynb" '+result_folder)